In [1]:
import os
import shutil
import json
import zipfile
import pyspark
import logging
import pandas as pd
from database_script import my_conn
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy.exc import SQLAlchemyError
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
# from selenium import webdriver
# from selenium.webdriver import chrome
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.chrome.options import Options
from data_ingestion_script import *
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
logger = logging.getLogger('Data_Processing')
logger.info('Data_Processing Script started')

spark = SparkSession.builder.getOrCreate()

[('male',), ('ix_male_index',)]


2024-02-26 17:21:24,874 - Data_Processing Script started


In [ ]:
logger.info("Running the requirements.txt file")
!pip install -r requirements.txt
logger.info("requirements.txt runned successfully")

2024-02-26 05:30:13,734 - Running the requirements.txt file

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
2024-02-26 05:30:15,391 - requirements.txt runned successfully


In [ ]:
try:
    # Data Ingestion  to correct path
    origin = 'C:/Users/Admin/Downloads/'
    target_directory = 'C:/Users/Admin/Downloads/procesing/Batch_pipeline_DataEngineering_task/'
    download_required_files(logger)
    logger.info("Ingestion of Data Completed successfully")

    file_in_origin = os.listdir(origin)
    # if ".crdownload" in  file_in_origin:
    #     logger.info("yes")
    # while ("odis_female_json.zip" not in file_in_origin) and ("odis_male_json.zip" not in file_in_origin):
    #     print("yes")
    #     logger.info("waiting for file to get downloaded")
    while ("odis_female_json.zip" not in file_in_origin) and ("odis_male_json.zip" not in file_in_origin):
        file_in_origin = os.listdir(origin)
        if ("odis_female_json.zip" in file_in_origin) and ("odis_male_json.zip" in file_in_origin):
            logger.info("waiting for file to get downloaded")
            break
    if ('LANDING' in os.listdir(f'{target_directory}')) and ('DOWNLOAD_PATH' in os.listdir(f'{target_directory}')):
        logger.info("LANDING PATH IS THERE")
        logger.info("DOWNLOAD PATH IS THERE")
    else :
        os.makedirs(f'{target_directory}LANDING')
        os.makedirs(f'{target_directory}DOWNLOAD_PATH')

    files_in_target_directory = os.listdir(target_directory+'DOWNLOAD_PATH')
    for file in file_in_origin:
        if (file.__contains__("odis_female_json") or file.__contains__("odis_male_json")) and (('odis_female_json.json' not in files_in_target_directory) and ('odis_male_json.json' not in files_in_target_directory)):
            shutil.copy(origin+file, target_directory)
    # for file_name in ['odis_female_json.zip', "odis_male_json.zip"]:
    #     with zipfile.ZipFile(f'{target_directory}{file_name}') as f:
except Exception as e:
    logging.error("Exception occurred", exc_info=True)

2024-02-26 17:18:03,711 - ====== WebDriver manager ======
2024-02-26 17:18:05,490 - Get LATEST chromedriver version for google-chrome
2024-02-26 17:18:05,685 - Get LATEST chromedriver version for google-chrome
2024-02-26 17:18:05,922 - Driver [C:\Users\Admin\.wdm\drivers\chromedriver\win64\122.0.6261.69\chromedriver-win32/chromedriver.exe] found in cache
2024-02-26 17:18:23,613 - Download of Male_Dataset Started
2024-02-26 17:18:25,826 - Download of Female_Dataset Started
2024-02-26 17:18:27,855 - Ingestion of Data Completed successfully
2024-02-26 17:18:27,855 - Exception occurred
Traceback (most recent call last):
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_7608\756284540.py", line 24, in <module>
    os.makedirs(f'{target_directory}DOWNLOAD_PATH')
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\os.py", line 225, in makedirs
    mkdir(name, mode)
FileExistsError: [WinError 183] Cannot create a file when that file al

In [ ]:
# os.makedirs(f'{target_directory}LANDING')
os.listdir()

['.git',
 '.venv',
 'database',
 'database_script.py',
 'DataSlush_ver1.ipynb',
 'data_ingestion_script.py',
 'download_path',
 'LICENSE',
 'README.md',
 'requirements.txt',
 '__pycache__']

In [ ]:
#Necessay Function
def get_batter_bowler_striker(column, column_need):
  try:
    if column_need.strip() == 'batter':
      return column['batter']
 
    elif column_need.strip() == 'bowler':
      return column['bowler']

    elif column_need.strip() == 'non_striker':
      return column['non_striker']
  except Exception as e:
    logging.error("Exception occurred", exc_info=True)



#Necessay Function
def get_run_ball_by_ball(column):
  try:

    runs_list = [-1]*3
    for scores in column.keys():
      if scores == 'runs':
        return column[scores]
      
  except Exception as e:
    logging.error("Exception occurred", exc_info=True)


def get_info_and_meta_data(all_information):
  try:
      for data in all_information.keys():
        if data == 'meta':
          meta_information = all_information['meta']
        elif data == 'info':
          info_information = all_information["info"]
      return meta_information, info_information
  except Exception as e:
    logging.error("Exception occurred", exc_info=True)

def get_necessary_information(data):
  try:
      date = data['dates'][0]

      if "city" in data.keys():
        city = data['city']
      else:
        city = "NULL"
      event_name = data['event']['name']

      if 'match_number' in data['event'].keys():
        match_number = data['event']['match_number']
      else:
        match_number = "NULL"
      gender = data['gender']

      if "winner" in data['outcome'].keys():

        winner_team = data['outcome']['winner']
      elif 'result' in data['outcome'].keys():

        winner_team = data['outcome']['result']

      if "winner" in data['outcome'].keys():
        if 'wickets' in data['outcome']['by'].keys():
          winned_by = f"{data['outcome']['by']['wickets']} wickets"
        elif 'runs' in data['outcome']['by'].keys():
          winned_by = f"{data['outcome']['by']['runs']} runs"
      else:
        winned_by = "NULL"
      team_1 = data['teams'][0]
      team_2 = data['teams'][1]
      return date, city, event_name, match_number, gender, winner_team, winned_by, team_1, team_2
  except Exception as e:
    logging.error("Exception occurred", exc_info=True)

#UDF created to get the run scored ball-by-ball
get_run_ball_by_ball_udf =  udf(lambda column: get_run_ball_by_ball(column), StringType())
#UDF created to get the batter name, striker name, bowler name
get_batter_bowler_striker_udf =  udf(lambda column, column_need: get_batter_bowler_striker(column, column_need), StringType())

In [ ]:
path = "C:/Users/Admin/Downloads/male_dataset"
all_the_files = os.listdir(path)
# print(all_the_files)
necessary_columns = []
for i in all_the_files:
  # print(f"/female_dataset/{i}")
  if i.endswith(".json"):
    with open(f"{path}/{i}","r") as file_obj:
      file_content = file_obj.read()
      # print(file_content)
      details = json.loads(file_content)
      meta_information, info_information = get_info_and_meta_data(details)
      date, city, event_name, match_number, gender, winner_team, winned_by, team_1, team_2 = get_necessary_information(info_information)
      necessary_columns = []
      temp_dict = {}
      for i in range(len(details['innings'][0]['overs'])):
        temp_dict['overs'] = i
        temp_dict['balls_per_over'] = details['innings'][0]['overs'][i]['deliveries']
        # print(student_details['innings'][0]['overs'][i]['deliveries'])
        necessary_columns.append(temp_dict)
        temp_dict = {}
      dataframe = spark.createDataFrame(necessary_columns)
      # dataframe.show(1000, False)
      dataframe = dataframe.select("overs", posexplode_outer(dataframe.balls_per_over))

      dataframe_new = dataframe.withColumn("BATTER", get_batter_bowler_striker_udf(col("col"), lit("batter")))
      dataframe_new = dataframe_new.withColumn("BOWLER", get_batter_bowler_striker_udf(col("col"), lit("bowler")))
      dataframe_new = dataframe_new.withColumn("NON_STRIKER", get_batter_bowler_striker_udf(col("col"), lit("non_striker")))
      dataframe_new = dataframe_new.withColumn("runs_scored_per_ball", get_run_ball_by_ball_udf(col("col")))
      dataframe_new = dataframe_new.select('*', lit(date).alias("MATCH_DATE"), lit(city).alias("MATCH_CITY"),\
                                           lit(event_name).alias("EVENT_NAME"),lit(match_number).alias("MATCH_NUMBER"),lit(gender).alias("GENDER"),\
                                           lit(winner_team).alias("WINNER_TEAM"),lit(winned_by).alias("WINNED_BY"),lit(team_1).alias("FIRST_TEAM"),lit(team_2).alias("SECOND_TEAM"))

      dataframe_new = dataframe_new.withColumn("runs_scored_per_ball", regexp_replace(col("runs_scored_per_ball"), "(\{extras=)|(total=)|(batter=)|(\})", "")).withColumn("EXTRAS_EARNED_PER_BALL", trim(split(col("runs_scored_per_ball"), ',').getItem(0))).withColumn("TOTAL_RUNS_PER_BALL", trim(split(col("runs_scored_per_ball"), ',').getItem(1))).withColumn("BATTER_SCORED_RUNS_PER_BALL", trim(split(col("runs_scored_per_ball"), ',').getItem(2)))
      dataframe_new = dataframe_new.select('overs','FIRST_TEAM', 'SECOND_TEAM','EVENT_NAME','MATCH_DATE','MATCH_CITY','MATCH_NUMBER','GENDER','WINNER_TEAM','WINNED_BY','BATTER','BOWLER','NON_STRIKER','BATTER_SCORED_RUNS_PER_BALL','TOTAL_RUNS_PER_BALL', 'EXTRAS_EARNED_PER_BALL')
      # dataframe_new.write.parquet("/output_folder/")\
      # dataframe_new.write.mode("append").format("parquet").save("/output_folder/")
      dataframe_new.show(10, False)
      dataframe_new = dataframe_new.toPandas()
      dataframe_new.to_sql(con = my_conn, name = 'male', if_exists='replace')
      # dataframe_new.show(10, Fals
      # result = my_conn.execute(text("select * from male where WINNED_BY NOT like '%runs%'"))
      # print(result.all())
      # break
   


+-----+----------+-----------+--------------------------------+----------+----------+------------+------+-----------+---------+---------+---------------+-----------+---------------------------+-------------------+----------------------+
|overs|FIRST_TEAM|SECOND_TEAM|EVENT_NAME                      |MATCH_DATE|MATCH_CITY|MATCH_NUMBER|GENDER|WINNER_TEAM|WINNED_BY|BATTER   |BOWLER         |NON_STRIKER|BATTER_SCORED_RUNS_PER_BALL|TOTAL_RUNS_PER_BALL|EXTRAS_EARNED_PER_BALL|
+-----+----------+-----------+--------------------------------+----------+----------+------------+------+-----------+---------+---------+---------------+-----------+---------------------------+-------------------+----------------------+
|0    |Australia |Pakistan   |Pakistan in Australia ODI Series|2017-01-13|Brisbane  |1           |male  |Australia  |92 runs  |DA Warner|Mohammad Amir  |TM Head    |0                          |extras=0           |{0                    |
|0    |Australia |Pakistan   |Pakistan in Australia 

2024-02-26 06:13:07,360 - KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "c:\Users\Admin\Downloads\procesing\Batch_pipeline_DataEngineering_task\.venv\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "c:\Users\Admin\Downloads\procesing\Batch_pipeline_DataEngineering_task\.venv\lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt
2024-02-26 06:13:07,448 - Closing down clientserver connection


KeyboardInterrupt: 

In [ ]:
result = my_conn.execute(text("select * from male where WINNED_BY = 'NULL'"))
print(result.all())

[]


In [ ]:
import zipfile
path = "C:/Users/Admin/Downloads/"

In [ ]:
print(os.getcwd())

C:\Users\Admin\Downloads\male_dataset


In [ ]:
import os
import shutil

# Providing the folder path
origin = 'C:/Users/Admin/Downloads/'
target_directory = 'C:/Users/Admin/Downloads/procesing/Batch_pipeline_DataEngineering_task/download_path/'
# female_directory = 'C:/Users/Admin/Downloads/procesing/Batch_pipeline_DataEngineering_task/download_path/female_folder'
# target = 'C:\Users\Lenovo\Downloads\Work TP'

# Fetching the list of all the files
files = os.listdir(origin)
files_in_target_directory = os.listdir(target_directory)
print(files)

for file in files:
    if (file.__contains__("odis_female_json") or file.__contains__("odis_male_json")) and (('odis_female_json.json' not in files_in_target_directory) and ('odis_male_json.json' not in files_in_target_directory)):
        shutil.copy(origin+file, target_directory)


# Fetching all the files to directory
# for file_name in files:
#    shutil.copy(origin+file_name, target+file_name)
# print("Files are copied successfully")

['111.png', 'all_csv.zip.crdownload', 'Anaconda3-2023.09-0-Windows-x86_64.exe', 'android-programming-tutorials-d10189445.html', 'android-studio-2023.1.1.27-windows.exe', 'archive.zip', 'bd912de1-d0de-43e4-b812-96302ec66965.csv', 'blender-3.5.0-windows-x64.msi', 'ChromeSetup.exe', 'connect_with_s3_usingboto3.ipynb', 'csv_format.csv', 'Data_engineer_gautam_kumar_resume_ver3-1.pdf', 'desktop.ini', 'DiscordSetup.exe', 'Docker Desktop Installer.exe', 'female_datset', 'fileInput (1).txt', 'fileInput (2).txt', 'fileInput (3).txt', 'fileInput (4).txt', 'fileInput (5).txt', 'fileInput (6).txt', 'fileInput (7).txt', 'fileInput.txt', "Gautam Kumar's Resume.pdf", 'gautam.kumar410305@gmail.com.pdf', 'Gautam_kumar_DataEngineer 4.pdf', 'Gautam_kumar_DataEngineer.docx', 'Git-2.40.0-64-bit.exe', 'GitHubDesktopSetup-x64.exe', 'hadoop', 'ideaIC-2023.3.3.exe', 'jdk-20_windows-x64_bin.exe', 'jdk-8u361-windows-x64.exe', 'Machine-Learning-with-Python-GFG-main.zip', 'machine_learning-linear_regression (1).zip